In [2]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt 
import os
!pip install Pillow
import seaborn as sn

from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import ensemble
from sklearn import tree
import xgboost as xgb

%pip install git+https://github.com/mwalmsley/galaxy_mnist.git
from galaxy_mnist import GalaxyMNIST

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [5]:
#  Import galaxyMNIST dataset
train_dataset = GalaxyMNIST(
    root='ML_DP/gal_mnist',
    download=True
)

test_dataset = GalaxyMNIST(
    root='ML_DP/gal_mnist',
    download=True,
    train=False
)

NameError: name 'GalaxyMNIST' is not defined

In [ ]:
images_train=train_dataset.data
y_train = train_dataset.targets
images_val=test_dataset.data
y_val = test_dataset.targets

print(images_train[0].shape)
nsamples , n , nx, ny = images_train.shape
d2_images_train = images_train.reshape((nsamples,n*nx*ny))
nsamples , n , nx, ny = images_val.shape
d2_images_val = images_val.reshape((nsamples,n*nx*ny))

# ***Gini***

In [ ]:

classifier_gini_nomax = tree.DecisionTreeClassifier(criterion='gini', random_state=0)
classifier_gini_nomax.fit(d2_images_train, y_train)


n_nodes = classifier_gini_nomax.tree_.node_count
print(n_nodes)
print(classifier_gini_nomax.tree_.max_depth)

In [ ]:
# vogliamo capire qual è il max depth ideale. facciamo un 5-fold cross-validation
values = np.arange(3,27,3) # il 27 viene dal passo prima. 
cv_scs = {}
for i in values:
    clf = tree.DecisionTreeClassifier(max_depth=i,random_state=0)
    cv_scs[i]= cross_val_score(clf, d2_images_train, y_train, cv=5, n_jobs=-1)

In [ ]:
error_score=[]
for i in cv_scs.values():
    error_score.append(1-i)
plt.plot(cv_scs.keys(),error_score)
plt.show()

In [ ]:
values_new = [6,7,8,9,10,11] # proviamo a vedere se tra 6 e 11 ci sono valori migliori
#from sklearn.model_selection import cross_val_score
cv_scs_new = {}
for i in values_new:
    clf = tree.DecisionTreeClassifier(max_depth=i,random_state=0)
    cv_scs_new[i]= cross_val_score(clf, d2_images_train, y_train, cv=5, n_jobs=-1)

In [ ]:
error_score_new=[]
for i in cv_scs_new.values():
    error_score_new.append(1-i)
plt.plot(cv_scs_new.keys(),error_score_new)
plt.show()

In [ ]:
classifier_gini = tree.DecisionTreeClassifier(criterion='gini', max_depth=9, random_state=0) # la max_depth viene dal passo precedente, ed è 9
classifier_gini.fit(d2_images_train, y_train) # fittiamo con le immagini del train

#vediamo il numero di nodi e la max depth attuale dell'albero
n_nodes = classifier_gini.tree_.node_count
print(n_nodes)
print(classifier_gini.tree_.max_depth)


In [ ]:
y_pred_gini = classifier_gini.predict(d2_images_val) # ora usiamo l albero per predire le labels delle immagini in test 

print('Model accuracy score with criterion gini index: {0:0.4f}'. format(accuracy_score(y_val, y_pred_gini)))

In [ ]:
# facciamo il check per eventuale overfit

y_pred_train_gini = classifier_gini.predict(d2_images_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train_gini)))

In [ ]:
plt.figure(figsize=(15,15))



tree.plot_tree(classifier_gini.fit(d2_images_train, y_train))

In [ ]:
cf_matrix = confusion_matrix(y_val, y_pred_gini)
classes = GalaxyMNIST.classes
df_cm = pd.DataFrame(cf_matrix 
                     , index = [i for i in classes], columns = [i for i in classes])
plt.figure(figsize = (10,6))
sn.heatmap(df_cm, annot=True, fmt='.0f')

# ***Entropy***

In [ ]:
#ATTENZIONE
# instantiate the DecisionTreeClassifier model with criterion entropy

classifier_entropy_nomax = tree.DecisionTreeClassifier(criterion='entropy', random_state=0) #max_depth=3
# fit the model
classifier_entropy_nomax.fit(d2_images_train, y_train)


In [ ]:
#vediamo il numero di nodi e la max depth attuale dell'albero
n_nodes = classifier_entropy_nomax.tree_.node_count
print(n_nodes)
print(classifier_entropy_nomax.tree_.max_depth)

In [ ]:
# vogliamo capire qual è il max depth ideal. facciamo un 5-fold cross-validation
values = np.arange(2,20,3) # il 20 viene dal passo prima. 

cv_scs = {}
for i in values:
    clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=i,random_state=0)
    cv_scs[i]= cross_val_score(clf, d2_images_train, y_train, cv=5, n_jobs=-1)

In [ ]:
error_score=[]
for i in cv_scs.values():
    error_score.append(1-i)
plt.plot(cv_scs.keys(),error_score)
plt.show()

In [ ]:
# vediamo se ci sono valori migliori attorno a 8
values_new = [6,7,8,9,10]
from sklearn.model_selection import cross_val_score
cv_scs_new = {}
for i in values_new:
    clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=i,random_state=0)
    cv_scs_new[i]= cross_val_score(clf, d2_images_train, y_train, cv=5, n_jobs=-1)

In [ ]:
error_score_new=[]
for i in cv_scs_new.values():
    error_score_new.append(1-i)
plt.plot(cv_scs_new.keys(),error_score_new)
plt.show()

In [ ]:
classifier_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=8, random_state=0) #max_depth vista dal passo precedente
# fit the model
classifier_entropy.fit(d2_images_train, y_train)

In [ ]:
#vediamo il numero di nodi e la max depth attuale dell'albero
n_nodes = classifier_entropy.tree_.node_count
print(n_nodes)
print(classifier_entropy.tree_.max_depth)

In [ ]:
y_pred_entropy = classifier_entropy.predict(d2_images_val)

print('Model accuracy score with criterion entropy index: {0:0.4f}'. format(accuracy_score(y_val, y_pred_entropy)))

y_pred_train_entropy = classifier_entropy.predict(d2_images_train)

print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train_entropy)))

In [ ]:
cf_matrix = confusion_matrix(y_val, y_pred_entropy)
classes = GalaxyMNIST.classes
df_cm = pd.DataFrame(cf_matrix 
                     , index = [i for i in classes], columns = [i for i in classes])
plt.figure(figsize = (10,6))
sn.heatmap(df_cm, annot=True, fmt= '.0f')

# ***Bagging***

In [ ]:
bagging_model = ensemble.BaggingClassifier(tree.DecisionTreeClassifier(max_depth = 9), n_estimators = 15, max_samples = 0.50, max_features = 0.50, random_state=42) # bootstrap, di default è true
bagging_model.fit(d2_images_train, y_train)

In [ ]:
y_train_predBag = bagging_model.predict(d2_images_train)

print(confusion_matrix(y_train, y_train_predBag))
print(accuracy_score(y_train, y_train_predBag))

In [ ]:
y_val_predBag = bagging_model.predict(d2_images_val)

print(confusion_matrix(y_val, y_val_predBag))
print(accuracy_score(y_val, y_val_predBag))

In [ ]:
cf_matrix = confusion_matrix(y_val, y_val_predBag)
classes = GalaxyMNIST.classes
df_cm = pd.DataFrame(cf_matrix 
                     , index = [i for i in classes], columns = [i for i in classes])
plt.figure(figsize = (10,6))
sn.heatmap(df_cm, annot=True, fmt= '.0f')

# ***XGBoosting***

In [ ]:
gino = 3 

xgb_model = xgb.XGBClassifier(objective='multi:softmax', tree_method='hist',  device='cuda', max_depth=3, random_state=42)
xgb_model.fit(d2_images_train, y_train)

In [ ]:
y_train_pred = xgb_model.predict(d2_images_train)

print(confusion_matrix(y_train, y_train_pred))
print(accuracy_score(y_train, y_train_pred))

In [ ]:
y_val_pred = xgb_model.predict(d2_images_val)

print(confusion_matrix(y_val, y_val_pred))
print(accuracy_score(y_val, y_val_pred))

In [ ]:
cf_matrix = confusion_matrix(y_val, y_val_pred)
classes = GalaxyMNIST.classes
df_cm = pd.DataFrame(cf_matrix 
                     , index = [i for i in classes], columns = [i for i in classes])
plt.figure(figsize = (10,6))
sn.heatmap(df_cm, annot=True, fmt= '.0f')